# Probabilistic Forecasting & Prediction Intervals

Point forecasts hide uncertainty. Probabilistic forecasts express **a distribution**
for future values, enabling risk-aware decisions.


## Quantiles and intervals

A forecast distribution yields **quantiles** $q_lpha$ such that:

\[P\left(y_{t+h} \le q_lphaight) = lpha\]

A two-sided $(1-lpha)$ prediction interval is:

\[\left[q_{lpha/2},\; q_{1-lpha/2}ight]\]


In [ ]:
import numpy as npimport pandas as pdimport plotly.graph_objects as gofrom statistics import NormalDistnp.random.seed(21)# Synthetic series with noisen = 120t = np.arange(n)y = 20 + 0.15 * t + 4 * np.sin(2 * np.pi * t / 12) + np.random.normal(0, 1.3, n)# Simple mean forecast: last seasonal cycleh = 12fh = np.arange(1, h + 1)forecast_idx = t[-1] + fhseasonal = 4 * np.sin(2 * np.pi * forecast_idx / 12)mean_forecast = (y[-12:].mean()) + seasonal# Estimate residual scale from recent historyresiduals = y[-36:] - (y[-36:].mean() + 4 * np.sin(2 * np.pi * t[-36:] / 12))sigma = residuals.std(ddof=1)# Build quantile fanquantiles = [0.05, 0.2, 0.35, 0.5, 0.65, 0.8, 0.95]q_forecasts = {}for q in quantiles:    z = NormalDist().inv_cdf(q)    q_forecasts[q] = mean_forecast + z * sigmafig = go.Figure()fig.add_trace(go.Scatter(x=t, y=y, mode="lines", name="Observed", line=dict(color="#2a3f5f")))# Fan chart: draw from outer to innerpalette = ["#fdebd0", "#f6ddcc", "#f5cba7", "#edbb99", "#e59866", "#dc7633"]pairs = [(0.05, 0.95), (0.2, 0.8), (0.35, 0.65)]for i, (low, high) in enumerate(pairs):    fig.add_trace(        go.Scatter(            x=np.concatenate([forecast_idx, forecast_idx[::-1]]),            y=np.concatenate([q_forecasts[high], q_forecasts[low][::-1]]),            fill="toself",            fillcolor=palette[i],            line=dict(color="rgba(0,0,0,0)"),            name=f"{int((high-low)*100)}% interval",        )    )fig.add_trace(    go.Scatter(        x=forecast_idx,        y=q_forecasts[0.5],        mode="lines+markers",        name="Median forecast",        line=dict(color="#ef553b"),    ))fig.update_layout(title="Probabilistic forecast fan chart", height=420)fig

## Calibration check (coverage)

If your 80% intervals are well calibrated, about 80% of true values should fall
inside them over many backtests. Coverage diagnostics help quantify miscalibration.


In [ ]:
# Simple coverage simulation with synthetic residuals
np.random.seed(22)

sim_n = 500
true = np.random.normal(0, 1.0, sim_n)
mu = np.zeros(sim_n)

sigma = 1.0
interval_80 = (
    NormalDist().inv_cdf(0.1) * sigma,
    NormalDist().inv_cdf(0.9) * sigma,
)

lower = mu + interval_80[0]
upper = mu + interval_80[1]
coverage = ((true >= lower) & (true <= upper)).mean()
coverage


## sktime mapping (practical pointers)

Many sktime forecasters implement probabilistic APIs:
- `predict_interval` for prediction intervals
- `predict_quantiles` for quantile forecasts
- `predict_var` for variance forecasts

Use these with `fh` to generate horizon-aligned uncertainty.
